In [15]:
import pandas as pd
import pymysql

# Step 1: Load the CSV file and parse dates
df = pd.read_csv('merged_data.csv', parse_dates=['Order Date', 'Birthday', 'Open Date'])
df.rename(columns={'Open Date': 'Store_Open_Date', 'Square Meters': 'Store_Square_Meters','Store_Country': 'Store_Country'}, inplace=True)
# Assuming 'UnitCost' and 'UnitPrice' are columns in merged_df


# Step 2: Connect to MySQL Database
db_user = 'root'
db_password = 'Dhilip@04'
db_host = 'localhost'
db_name = 'dataspark'

# Connect without specifying the database first
connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password
)

cursor = connection.cursor()

# Create the database if it doesn't exist
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name};")
cursor.execute(f"USE {db_name};")

# Step 3: Create the table if not exists, including Store_Country and Store_State
# Step 3: Create the table if not exists, increase State_Code VARCHAR length
create_table_query = """
CREATE TABLE IF NOT EXISTS merged_data (
    Order_Number INT,
    Line_Item INT,
    Order_Date DATE,
    Delivery_Date VARCHAR(255),
    CustomerKey INT,
    StoreKey INT,
    ProductKey INT,
    Quantity INT,
    Currency_Code VARCHAR(10),
    Gender VARCHAR(10),
    Name VARCHAR(255),
    City VARCHAR(255),
    State_Code VARCHAR(20),  # Updated length to 20
    State VARCHAR(255),
    Zip_Code VARCHAR(10),
    Country VARCHAR(255),
    Continent VARCHAR(255),
    Birthday DATE,
    Product_Name VARCHAR(255),
    Brand VARCHAR(255),
    Color VARCHAR(50),
    Unit_Cost_USD FLOAT,
    Unit_Price_USD FLOAT,
    Actual_Unitcost_In_USD FLOAT,
    Actual_Unitprice_In_USD FLOAT,
    SubcategoryKey INT,
    Subcategory VARCHAR(255),
    CategoryKey INT,
    Category VARCHAR(255),
    Store_Square_Meters FLOAT,
    Store_Open_Date DATE,
    Store_Country VARCHAR(255),  
    Store_State VARCHAR(255)
);
"""
cursor.execute(create_table_query)

cursor.execute(create_table_query)

# Step 4: Insert data into the table row by row, including Store_Country and Store_State
insert_query = """
INSERT INTO merged_data (
    Order_Number, Line_Item, Order_Date, Delivery_Date, CustomerKey, StoreKey, ProductKey, Quantity, 
    Currency_Code, Gender, Name, City, State_Code, State, Zip_Code, Country, Continent, Birthday, 
    Product_Name, Brand, Color, Unit_Cost_USD, Unit_Price_USD, Actual_Unitcost_In_USD,
    Actual_Unitprice_In_USD,SubcategoryKey, Subcategory, 
    CategoryKey, Category, Store_Square_Meters, Store_Open_Date, Store_Country, Store_State
) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s);
"""

# Iterate over rows and insert them into the database
for index, row in df.iterrows():
    # Truncate State_Code to 20 characters
    row_values = (
        row['Order Number'] if pd.notna(row['Order Number']) else None,
        row['Line Item'] if pd.notna(row['Line Item']) else None,
        row['Order Date'].strftime('%Y-%m-%d') if pd.notna(row['Order Date']) else None,
        row['Delivery Date'] if pd.notna(row['Delivery Date']) else "Not Delivered",
        row['CustomerKey'] if pd.notna(row['CustomerKey']) else None,
        row['StoreKey'] if pd.notna(row['StoreKey']) else None,
        row['ProductKey'] if pd.notna(row['ProductKey']) else None,
        row['Quantity'] if pd.notna(row['Quantity']) else None,
        row['Currency Code'] if pd.notna(row['Currency Code']) else None,
        row['Gender'] if pd.notna(row['Gender']) else None,
        row['Name'] if pd.notna(row['Name']) else None,
        row['City'] if pd.notna(row['City']) else None,
        str(row['State Code'])[:20] if pd.notna(row['State Code']) else None,  # Truncate to 20 chars
        row['State'] if pd.notna(row['State']) else None,
        row['Zip Code'] if pd.notna(row['Zip Code']) else None,
        row['Country'] if pd.notna(row['Country']) else None,
        row['Continent'] if pd.notna(row['Continent']) else None,
        row['Birthday'].strftime('%Y-%m-%d') if pd.notna(row['Birthday']) else None,
        row['Product Name'] if pd.notna(row['Product Name']) else None,
        row['Brand'] if pd.notna(row['Brand']) else None,
        row['Color'] if pd.notna(row['Color']) else None,
        row['Unit Cost USD'] if pd.notna(row['Unit Cost USD']) else None,
        row['Unit Price USD'] if pd.notna(row['Unit Price USD']) else None,
        row['Actual_Unitcost_In_USD'] if pd.notna(row['Unit Price USD']) else None,
        row['Actual_Unitprice_In_USD'] if pd.notna(row['Unit Price USD']) else None,
        row['SubcategoryKey'] if pd.notna(row['SubcategoryKey']) else None,
        row['Subcategory'] if pd.notna(row['Subcategory']) else None,
        row['CategoryKey'] if pd.notna(row['CategoryKey']) else None,
        row['Category'] if pd.notna(row['Category']) else None,
        row['Store_Square_Meters'] if pd.notna(row['Store_Square_Meters']) else None,
        row['Store_Open_Date'].strftime('%Y-%m-%d') if pd.notna(row['Store_Open_Date']) else None,
        row['Store_Country'] if pd.notna(row['Store_Country']) else None,
        row['Store_State'] if pd.notna(row['Store_State']) else None
)

    
    
    
   

    # Execute the insert query
    cursor.execute(insert_query, row_values)

# Commit the transaction to save the data
connection.commit()

# Close the connection
cursor.close()
connection.close()

print("Data successfully uploaded to MySQL database.")


Data successfully uploaded to MySQL database.
